<div style="text-align: center; background-color: #ADD8E6; font-family: 'Times New Roman', serif; color: white; padding: 20px; font-size: 30px; font-weight: bold; border-radius: 10px; box-shadow: 0px 6px 15px rgba(0, 0, 0, 0.2); transition: all 0.3s ease;">
    THE ANALYSIS OF THE MOBILE PHONE MARKET AT MOBILE CITY STORE
</div>
<style>
    div:hover {
        transform: scale(1.05);
        box-shadow: 0px 10px 20px rgba(0, 0, 0, 0.3);
    }
</style>


---

<div style="text-align: center; background-color: #ADD8E6; font-family: 'Times New Roman', serif; color: white; padding: 25px; font-size: 20px; font-weight: bold; border-radius: 10px; box-shadow: 0px 6px 15px rgba(0, 0, 0, 0.2); transition: all 0.3s ease;">
    DATA COLLECTION
</div>
<style>
    div:hover {
        opacity: 0.8;
    }
</style>

---

# Thư viện

Sử dụng Selenium kết hợp với BeautySoup để thu thập dữ liệu

In [1]:
from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from bs4 import BeautifulSoup
import json
from urllib.parse import urlparse
import csv

import pandas as pd

# Lấy thời gian hiện tại và cài đặt trình duyệt

In [2]:
# Get current time
current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Print the current time
print("Thời gian bắt đầu thu thập:", current_time)

Thời gian bắt đầu thu thập: 2024-12-18 07:28:23


In [3]:
# Tạo instance của trình duyệt
driver = webdriver.Chrome()

# Phóng to trình duyệt
driver.maximize_window()

# Lấy đường dẫn đến từng trang điện thoại chi tiết từ file `links.csv`

In [4]:
data = pd.read_csv("../data/links.csv")
links = data.links.to_list()

In [5]:
def is_valid_url(url):
    """Check if the URL is valid."""
    parsed = urlparse(url)
    return bool(parsed.scheme and parsed.netloc)

# Thu thập các thuộc tính của từng sản phẩm

In [6]:
product_data = []
counter = 0  
wait = WebDriverWait(driver, 10)

> **Chạy ô dưới đây nhiều lần cho đến khi nào thu thập được hết toàn bộ sản phẩm thì thôi**

Vì có trường hợp chạy mà mạng bị ngắt giữa chừng, hoặc driver bị crash, hoặc tự động ngắt kết nối notebook trong VSCode hay Google Colab,...

Ở đây, ô bên dưới đã được chạy 2 lần thì hoàn thành thu thập (`Từ cell [6] đến cell [8]`).

In [8]:
for i in range(counter, len(links)):
    links[i] = links[i].strip()  # Xóa khoảng trắng đầu và cuối
    if not is_valid_url(links[i]):
        print(f"Invalid URL: {links[i]}")
        continue  # Skip invalid URLs
        
    driver.get(links[i])
    # time.sleep(3)  # Chờ trang tải

    # try:
    #     # Wait until the element with the text "TƯ VẤN MÁY CŨ" appears
    #     element = WebDriverWait(driver, 10).until(
    #         EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'TƯ VẤN MÁY CŨ')]"))
    #     )
    #     # print("Element appeared!")
    # except:    
    #     print("Element did not appear within the given time")   
    
    # try:
    #     # button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'altumcode-close')))
    #     button = driver.find_elements(By.XPATH, '//button[@class="altumcode-close"]')
    #     for b in button:
    #         b.click()
    # except:
    #     button = None

    # Lấy HTML trang hiện tại
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Thu thập tên sản phẩm
    try:
        product_title = soup.find("h1", class_="title").text.strip()
    except AttributeError:
        product_title = "Không có tên sản phẩm"

    # Thu thập hãng điện thoại
    second_li = driver.find_element(By.XPATH, '//ul[@itemscope]/li[2]/a/span')
    brand = second_li.text.strip()

    # Lấy thông tin về thời gian bảo hành
    try:
        warranty_text = soup.find('span', class_='warranty-content-default').get_text(strip=True)
    except AttributeError:
        warranty_text = "Không có thông tin bảo hành"

    # Lấy thông số kỹ thuật
    try:
        specs_table = soup.find('div', class_='product-info-content').find('table')
        specs = {}
        if specs_table:
            rows = specs_table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 2:
                    specs[cols[0].text.strip()] = cols[1].text.strip()
    except AttributeError:
        specs = {"Không có thông số kỹ thuật": "N/A"}

    # Thu thập thông tin đánh giá
    try:
        rating = soup.find('div', class_='comment-vote__star-number').get_text(strip=True)
    except AttributeError:
        rating = "Không có đánh giá"

    # Thu thập thông tin số lượt đánh giá và hỏi đáp
    try:
        total_reviews = soup.find('div', class_='comment-vote__star-total').get_text(strip=True)
        total_reviews = total_reviews.replace("đánh giá và hỏi đáp", "").strip()
    except AttributeError:
        total_reviews = "Không có thông tin đánh giá và hỏi đáp"

    ##### Màu sắc + Phiên bản bộ nhớ + Giá tương ứng

    color_storage_price = []
    color_items = driver.find_elements(By.CLASS_NAME, "color-item.attribute-item")
    storage_items = driver.find_elements(By.CLASS_NAME, "storage-item.attribute-item")

    if len(color_items) == 0 and len(storage_items) == 0:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        try:
            price_new = soup.find("p", class_="price").text.strip()
        except AttributeError:
            price_new = "Không có giá mới"
        try:
            price_old = soup.find("p", class_="price-old").text.strip()
        except AttributeError:
            price_old = "Không có giá cũ"
        color_storage_price.append(["", "", price_new, price_old])
    elif len(color_items) == 0:
        for storage in storage_items:
            # storage.click()
            while True:
                try:
                    storage.click()
                    break
                except:
                    continue
            storage_name = storage.text.strip()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            try:
                price_new = soup.find("p", class_="price").text.strip()
            except AttributeError:
                price_new = "Không có giá mới"
            try:
                price_old = soup.find("p", class_="price-old").text.strip()
            except AttributeError:
                price_old = "Không có giá cũ"
            color_storage_price.append(["", storage_name, price_new, price_old])
    elif len(storage_items) == 0:
        for color in color_items:
            # color.click()
            while True:
                try:
                    color.click()
                    break
                except:
                    continue
            color_name = color.get_attribute("data-title")
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            try:
                price_new = soup.find("p", class_="price").text.strip()
            except AttributeError:
                price_new = "Không có giá mới"
            try:
                price_old = soup.find("p", class_="price-old").text.strip()
            except AttributeError:
                price_old = "Không có giá cũ"
            color_storage_price.append([color_name, "", price_new, price_old])
    else:
        for color in color_items:
            for storage in storage_items:
                # color.click()
                # storage.click()
                while True:
                    try:
                        color.click()
                        storage.click()
                        break
                    except:
                        continue
                color_name = color.get_attribute("data-title")
                storage_name = storage.text.strip()
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                try:
                    price_new = soup.find("p", class_="price").text.strip()
                except AttributeError:
                    price_new = "Không có giá mới"
                try:
                    price_old = soup.find("p", class_="price-old").text.strip()
                except AttributeError:
                    price_old = "Không có giá cũ"    
                color_storage_price.append([color_name, storage_name, price_new, price_old])

    # # Thu thập màu sắc
    # try:
    #     color_items = soup.find_all("div", class_="color-item")
    #     colors = [color['data-title'] for color in color_items]
    # except AttributeError:
    #     colors = ["Không có màu sắc"]

    # # Lấy thông tin về các phiên bản bộ nhớ
    # try:
    #     storage_items = soup.find_all('div', class_='storage-item attribute-item')
    #     storage_options = [item.text.strip() for item in storage_items]
    # except AttributeError:
    #     storage_options = ["Không có phiên bản bộ nhớ"]

    # # Thu thập giá sản phẩm (giá mới và giá cũ)
    # try:
    #     price_new = soup.find("p", class_="price").text.strip()
    # except AttributeError:
    #     price_new = "Không có giá mới"

    # try:
    #     price_old = soup.find("p", class_="price-old").text.strip()
    # except AttributeError:
    #     price_old = "Không có giá cũ"

    #####

    product_data.append({
        "Tên sản phẩm": product_title,
        "Loại điện thoại": brand,
        # "Giá mới": price_new,
        # "Giá cũ": price_old,
        # "Màu sắc": ", ".join(colors),
        # "Các phiên bản bộ nhớ": ", ".join(storage_options),
        "Màu sắc - Phiên bản bộ nhớ - Giá tương ứng": color_storage_price,
        "Thời gian bảo hành": warranty_text,
        "Thông số kỹ thuật": json.dumps(specs, ensure_ascii=False),
        "Đánh giá": rating,
        "Số lượt đánh giá và hỏi đáp": total_reviews,
        "Đường dẫn": links[i],
    })
    counter +=1
    # if counter % 100 == 0:
    #     print(f"Đã thu thập thành công {counter} sản phẩm")
    print(f"Đã thu thập: {counter}/{len(links)} sản phẩm", end="\r")

# In thông báo khi hoàn thành
# print("Hoàn thành!!!")

In [9]:
# Get current time
current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Print the current time
print("Thời gian kết thúc thu thập:", current_time)

Thời gian kết thúc thu thập: 2024-12-18 09:17:23


# Lưu dữ liệu thu thập được vào file `raw_data.csv`

In [10]:
# Lưu dữ liệu vào file CSV
with open('../data/raw_data.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = product_data[0].keys()  # Lấy các tên trường từ phần tử đầu tiên
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for data in product_data:
        writer.writerow(data)

# Đóng trình duyệt
driver.quit()

Dataset này bao gồm các thuộc tính:
- `Index`: số thứ tự của đường dẫn đến từng điện thoại trên trang web MobileCity.
- `Tên sản phẩm`: tên của từng điện thoại.
- `Loại điện thoại`: chứa loại điện thoại được phân loại theo trang web MobileCity.
- `Màu sắc - Phiên bản bộ nhớ - Giá tương ứng`: là một danh sách các mẫu điện thoại của cùng một tên điện thoại, mỗi phần tử trong danh sách cũng là một danh sách bao gồm màu sắc, phiên bản bộ nhớ và giá tương ứng của mẫu điện thoại đó.
- `Thời gian bảo hành`: chứa thông tin bảo hành của điện thoại (đơn vị tháng/năm)
- `Thông số kỹ thuật`: là một từ điển với các giá trị tương ứng với các khóa sau: Màn hình, Hệ điều hành, Camera sau, Camera trước, CPU, RAM, Bộ nhớ trong, Thẻ SIM, Dung lượng pin, Thiết kế.
- `Đánh giá`: là đánh giá của điện thoại trên trang web MobileCity (từ 0-5 sao) ứng với mỗi tên điện thoại.
- `Số lượt đánh giá và hỏi đáp`: là số lượng đánh giá và hỏi đáp trên trang web MobileCity ứng với mỗi tên điện thoại.
- `Đường dẫn`: chứa đường dẫn đến tên điện thoại tương ứng trên trang web MobileCity.

Tổng cộng có `1444` sản phẩm, tương ứng `1444` tên điện thoại khác nhau.